In [1]:
using Kuber
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


## Multithreading

Start julia with JULIA_NUM_THREADS environment set

In [2]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "env": [{
                       "name": "JULIA_NUM_THREADS",
                       "value": "4"
                   }],
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(Threads.nthreads()); sleep(5); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "4"
                       },
                       "requests": {
                           "cpu": "4"
                       }
                   }
               }]
           }
       }""");

In [3]:
pod = put!(ctx, julia_pod);

In [4]:
;kubectl logs julia-pod

4
4


In [5]:
delete!(ctx, pod);

## Multiprocerssing

Use Julia Distributed package

In [6]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "using Distributed; addprocs(4); while true; println(nworkers()); sleep(5); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "4"
                       },
                       "requests": {
                           "cpu": "4"
                       }
                   }
               }]
           }
       }""");

In [7]:
pod = put!(ctx, julia_pod);

In [8]:
;kubectl logs julia-pod

4
4


In [9]:
delete!(ctx, pod);